# Decision Tree Classifier

In [5]:
# Useful imports
import numpy as np
import pandas as pd
import matplotlib

## a) Model

In [43]:
class DecisionTreeClassifier:

    def __init__(self, depth=30, max_samples_leaves=50): # absurb values bcs not subject of the assignment ;)
        self.depth = depth
        self.max_samples_leaves = max_samples_leaves

        self.count_depth = 0
        self.count_max_samples_leaves = 0

        self.tree = None

    def fit(self, X, y):
        self.nb_features = X.shape[1]
        self.count_depth = 0
        self.count_max_samples_leaves = 0

        # Start the recursive alg
        self.tree = self._algorithm(X, y)

        return 0

    # Multiclass gini index
    def _gini_index(self, y):
        total = len(y)
        unique, counts = np.unique(y, return_counts=True)
        counter = dict(zip(unique, counts))
        # {0: 7, 1: 4, 2: 1, 3: 2, 4: 1}

        gini_index = 1
        for i in counter:
            prob = counter[i] / total
            gini_index -= (prob ** 2)
        return gini_index

    def _calc_weighted_average_impurity(self, left, right):
        gini_indexe_left = self._gini_index(left[:, -1]) if len(left) > 0 else 0
        gini_indexe_right = self._gini_index(right[:, -1]) if len(right) > 0 else 0

        total = len(left) + len(right)
        average = ((len(left) / total) * gini_indexe_left) + ((len(right) / total) * gini_indexe_right)
        return average

    def _get_most_frequency_class(self, data):
        values, counts = np.unique(data, return_counts=True)
        return values[counts.argmax()]

    # The assignment 2 said that when need to choose the mean
    def _get_threshold(self, feature_index, data):
        if (isinstance(data[:, feature_index][0], str)):
            return self._get_most_frequency_class(data[:, feature_index])
        else:
            # Calculate the mean only for the feature indicated
            return np.mean(data[:, feature_index])

    # Index is the feature index
    def _test_split(self, index, value, dataset):
        left, right = [], []

        for row in dataset:

            if isinstance(value, str):
                if row[index] == value:
                    left.append(row)
                else:
                    right.append(row)
            else:
                if row[index] < value:
                    left.append(row)
                else:
                    right.append(row)

        return [np.array(left), np.array(right)]

    # Get the index of the best feature
    def _get_best_feature_to_split_on(self, dataset):

        min_average_impurity = 999
        feature_index = 0

        # Trying all features
        for i in range(self.nb_features):
            threshold = self._get_threshold(i, dataset)
            split_arr = self._test_split(i, threshold, dataset)
            average_impurity = self._calc_weighted_average_impurity(split_arr[0], split_arr[1])

            if average_impurity < min_average_impurity:
                min_average_impurity = average_impurity
                feature_index = i

        return feature_index

    # The Decision Tree multi-class classifier algorithm
    def _algorithm(self, X, y):
        dataset = np.column_stack([X, y])

        if (self._homogenuous(dataset) or self.count_depth == self.depth or self.count_max_samples_leaves == self.max_samples_leaves):
            self.count_max_samples_leaves += 1
            # Stop the recursive
            return {"label": self._labelize(dataset), "threshold": None, "left": None, "right": None}

        index_feature = self._get_best_feature_to_split_on(dataset)
        threshold = self._get_threshold(index_feature, dataset)
        split_arr = self._test_split(index_feature, threshold, dataset)
        
        try:
            left, right = split_arr
            left_X, left_y = left[:, :-1], left[:, -1]
            right_X, right_y = right[:, :-1], right[:, -1]
            self.count_depth += 1
        except:
            print(left, right)

        left_tree = self._algorithm(left_X, left_y)
        right_tree = self._algorithm(right_X, right_y)

        return {"index": index_feature, "threshold": threshold, "left": left_tree, "right": right_tree}

    def _homogenuous(self, dataset):
        y = dataset[:, -1]
        total = len(y)
        unique, counts = np.unique(y, return_counts=True)
        counter = dict(zip(unique, counts))
        # {0: 7, 1: 4, 2: 1, 3: 2, 4: 1}

        max_prob = 0
        for i in counter:
            prob = counter[i] / total

            if prob > max_prob:
                max_prob = prob

        # If one class has 90% frequency, return this class
        if max_prob > 0.90:
            return True
        return False

    # Return the value of the most frequent label
    def _labelize(self, dataset):
        y = dataset[:, -1]
        values, counts = np.unique(y, return_counts=True)

        ind = np.argmax(counts)
        return values[ind]

    def predict(self, X):
        predictions = []
        for row in X:
            predictions.append(self._predict_one(row, self.tree))
        return predictions
    
    # Go through linked node
    def _predict_one(self, row, tree):
        if tree["left"] is None and tree["right"] is None:
            return tree["label"]
        
        if row[tree["index"]] < tree["threshold"]:
            return self._predict_one(row, tree["left"])
        else:
            return self._predict_one(row, tree["right"])

## b) Train and Test model through different datasets

Use 2 useful functions from assignment 1

In [23]:
def split_tmls_to_X_and_y(data, label_to_remove = None):
    new_data = data.sample(frac = 1) # Suffle rows of the data set
    if (not(label_to_remove is None)):
        new_data = new_data[new_data["class"] != label_to_remove] # Drop class
    X = new_data.iloc[:, 0:-1]#.apply(pd.to_numeric, errors='coerce')
    X = (
        X.apply(pd.to_numeric, errors='coerce', downcast='float')
            .fillna(X)
    )
    y = new_data.iloc[:, -1]
    return X, y

In [8]:
# Function to calculate the accuracy of the model
def accuracy_score(y_true, y_pred):
    # Number of correct predictions
    correct = np.sum(y_true == y_pred)

    total = len(y_true)
    accuracy = correct / total
    return accuracy

Recode the `train_test_split` function from sklearn

In [9]:
def my_train_test_split(X, y, test_size=0.33):
    # This function supposes that we already have randomized rows
    size = X.shape[0]
    index = int(size - (size * test_size))

    X = X.to_numpy()
    y = y.to_numpy()

    return X[:index], X[index:], y[:index], y[index:]

Finally, we can setup a generic environment for tmls datasets

*Note: cancer.csv has been formated manually into cancer.tmls*

In [46]:
np.random.seed(1234) # Set initial random seed (good to always do)


for dataset_name in ["iris", "wine", "cancer", "adult"]:
    file_path = dataset_name + ".tmls"

    print(dataset_name + " dataset")

    for i in range(5):
        # Open dataset
        df_dataset = pd.read_csv(file_path)
        df_dataset = df_dataset.dropna() # Remove NaN
        data = df_dataset.drop(0) # Remove first row
        X, y = split_tmls_to_X_and_y(data)

        # Split data
        X_train, X_test, y_train, y_test = my_train_test_split(X, y, test_size=0.33)

        dt = DecisionTreeClassifier()
        dt.fit(X_train, y_train)
        pred = dt.predict(X_test)
        accuracy = accuracy_score(y_test, pred)
        print("Run " + str(i + 1) + " - Accuracy ", accuracy)

    print("\n")

iris dataset
Run 1 - Accuracy  0.94
Run 2 - Accuracy  1.0
Run 3 - Accuracy  0.96
Run 4 - Accuracy  0.98
Run 5 - Accuracy  0.92


wine dataset
Run 1 - Accuracy  0.9152542372881356
Run 2 - Accuracy  0.8983050847457628
Run 3 - Accuracy  0.8813559322033898
Run 4 - Accuracy  0.9152542372881356
Run 5 - Accuracy  0.8813559322033898


cancer dataset
Run 1 - Accuracy  0.9361702127659575
Run 2 - Accuracy  0.9308510638297872
Run 3 - Accuracy  0.9095744680851063
Run 4 - Accuracy  0.9095744680851063
Run 5 - Accuracy  0.9308510638297872


adult dataset
Run 1 - Accuracy  0.7402283161682591
Run 2 - Accuracy  0.7371261943169127
Run 3 - Accuracy  0.7406005707904206
Run 4 - Accuracy  0.7335897754063779
Run 5 - Accuracy  0.737808661124209


